In [ ]:
# Plot a 3D quiver plot
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt
import numpy as np

def myshrink(X,Y,Z,sh):
    xmin = np.min(X)
    xmax = np.max(X)
    ymin = np.min(Y)
    ymax = np.max(Y)
    zmin = np.min(Z)
    zmax = np.max(Z)
    zmin2 = zmin - sh * zmin
    zmax2 = zmax + sh * zmax
    xmin2 = xmin - sh * xmin
    xmax2 = xmax + sh * xmax
    ymin2 = ymin + sh * ymax
    ymax2 = ymax + sh * ymax
    
    return xmin,xmax,ymin,ymax,zmin,zmax

fig = plt.figure(figsize=(8,4),dpi=120)
ax = plt.subplot(111, projection="3d")
#ax = fig.gca(projection='3d')

X = stars['ra']
Y = stars['dec']
Z = stars['parallax']
pmra = stars['pmra']
pmdec = stars['pmdec']
parallax = stars['parallax']

# velocities
vx = []
vy = []
vz = []

# compute the velocity vectors
vx, vy, vz = compute_3d_velocities(ra,dec,pmra,pmdec,parallax)
veclength = np.sqrt(vx**2 + vy**2 + vz**2)

xmin,xmax,ymin,ymax,zmin,zmax = myshrink(vx,vy,vz,.05)
print(xmin,xmax,ymin,ymax,zmin,zmax)

ax.set_xlim3d(xmin, -xmin)
ax.set_ylim3d(ymin,-ymin)
ax.set_zlim3d(zmin,-zmin)
ax.view_init(60,45)


#ax.quiver(X, Y, Z, vx, vy, vz, length=0.02, normalize=True)
ax.quiver(0,0,0, vx, vy, vz,  length=10,normalize=False)

plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations


fig = plt.figure(figsize=(8,4),dpi=120)

ax = fig.gca(projection='3d')
#ax.set_aspect("equal")

xmin,xmax,ymin,ymax,zmin,zmax = myshrink(vx,vy,vz,.05)
print(xmin,xmax,ymin,ymax,zmin,zmax)

ax.set_xlim3d(xmin/10, xmax/10)
ax.set_ylim3d(ymin/10,ymax/10)
ax.set_zlim3d(zmin/10, zmax/10)
ax.view_init(40,45)



# draw sphere
u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
x = np.cos(u)*np.sin(v)
y = np.sin(u)*np.sin(v)
z = np.cos(v)
#ax.plot_wireframe(x, y, z, color="r")

# draw a point
ax.scatter([0], [0], [0], color="g", s=10)

# draw a vector
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d


class Arrow3D(FancyArrowPatch):

    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

i = 0
for i in range(star_count):
    point1 = [0,vx[i]]
    point2 = [0,vy[i]]
    point3 = [0,vz[i]]
    a = Arrow3D(point1, point2, point3, mutation_scale=3,lw=1, arrowstyle="-|>", color="k")
    ax.add_artist(a)
plt.show()




In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import norm

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
origin = np.array([0, 0, 0])
#axis and radius

def plot_cyl(ax, p0, p1, R):
    # p0 = np.array([1, 3, 2])
    # p1 = np.array([8, 5, 9])
    # R = .1
    #vector in direction of axis
    v = p1 - p0
    #find magnitude of vector
    mag = norm(v)
    #unit vector in direction of axis
    v = v / mag
    #make some vector not in the same direction as v
    not_v = np.array([1, 0, 0])
    if (v == not_v).all():
        not_v = np.array([0, 1, 0])
    #make vector perpendicular to v
    n1 = np.cross(v, not_v)
    #normalize n1
    n1 /= norm(n1)
    #make unit vector perpendicular to v and n1
    n2 = np.cross(v, n1)
    #surface ranges over t from 0 to length of axis and 0 to 2*pi
    t = np.linspace(0, mag, 10)
    theta = np.linspace(0, 2 * np.pi, 10)
    #use meshgrid to make 2d arrays
    t, theta = np.meshgrid(t, theta)
    #generate coordinates for surface
    X, Y, Z = [p0[i] + v[i] * t + R * np.sin(theta) * n1[i] + R * np.cos(theta) * n2[i] for i in [0, 1, 2]]
    ax.plot_surface(X, Y, Z)

p0 = np.array([0,0,0])
p1 = np.array([0, 0, 10000])
R = .01
#plot_cyl(ax,p0,p1,.1)

for i in range(star_count):
    p1 = np.array([vx[i]/10, vy[i]/10, vz[i]/10])
    plot_cyl(ax,p0,p1,1)
#plot axis
ax.plot(*zip(p0, p1), color = 'red')
ax.set_xlim(-200, 200)
ax.set_ylim(-200, 200)
ax.set_zlim(-200, 200)


#ax.set_ylim(0, 10)
#ax.set_zlim(0, 10)
plt.show()